Logs 
- [2025/08/07]    
  Get 31 bonds from the list of bonds in `./data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp`

- [2025/08/09]    
  Store all parameters and variables into `var_params_31bonds_dict`. 

In [243]:
import pandas as pd
import numpy as np
import re
import json

import matplotlib.pyplot as plt

from docplex.mp.model_reader import ModelReader

In [2]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
  "font.size": 16,
  "grid.alpha": 0.25})

In [3]:
lp_file_path = "./data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp"

model = ModelReader.read(lp_file_path)

In [4]:
binary_var_names = []
for var in model.iter_variables():
  if var.is_binary() and var.name is not None:
    var_name = var.name.split("_")[1]

    binary_var_names.append(var_name)

binary_var_names, len(binary_var_names)

(['020002BJ9',
  '026874DS3',
  '081437AT2',
  '097023CJ2',
  '13645RAD6',
  '13645RBF0',
  '14448CBC7',
  '15135BAW1',
  '21871XAS8',
  '24422EWZ8',
  '24422EXP9',
  '314353AA1',
  '36166NAK9',
  '438516CM6',
  '443201AC2',
  '444859BR2',
  '444859BV3',
  '444859BY7',
  '444859CA8',
  '45687VAB2',
  '539830CD9',
  '540424AT5',
  '56501RAN6',
  '655844CR7',
  '655844CT3',
  '75513EAD3',
  '759351AP4',
  '760759BA7',
  '760759BC3',
  '907818FX1',
  '91324PEJ7'],
 31)

Read the spreadsheet `data_assets_dictionary`

In [5]:
df = pd.read_excel("./data/1/data_assets_dictionary.xlsx", parse_dates=["maturity"])
df = df.drop(["Unnamed: 0"], axis=1)
df

,isin,ccy,assetId,strategyName,secGroup,secType,cpn,posSource,baseCcy,krd10y,...,Large Issuer_new,issuerTicker_new,sodraw.filterLevel1_new,sodraw.filterLevel3_new,sodraw.filterLevel4_new,security.elements.creditQualityBuckets_new,security.elements.normalizedRating_new,sodraw.filterLevel2_new,assetId_new,main_key
0,US001055BJ00,USD,001055BJ0,Unassigned,BND,CORP,3.6,EOD,USD,0.000000,...,N,AFL,Investment Grade Credit,Insurance,Life Insurance,A,A,Financial,001055BJ0,VCIT | VCIT (bench)
1,US001084AS13,USD,001084AS1,Unassigned,BND,CORP,5.8,EOD,USD,4.436857,...,N,AGCO,Investment Grade Credit,Capital Goods,Construction Machinery,BBB,BBB,Industrial,001084AS1,VCIT | VCIT (bench)
2,US00108WAM29,USD,00108WAM2,Unassigned,BND,CORP,2.1,EOD,USD,0.000000,...,N,AEP,Investment Grade Credit,Electric,Other,BBB,BBB,Utilities,00108WAM2,VCIT | VCIT (bench)
3,US00108WAP59,USD,00108WAP5,Unassigned,BND,CORP,4.7,EOD,USD,0.828971,...,N,AEP,Investment Grade Credit,Electric,Other,BBB,BBB,Utilities,00108WAP5,VCIT | VCIT (bench)
4,US00108WAR16,USD,00108WAR1,Unassigned,BND,CORP,5.4,EOD,USD,2.746458,...,N,AEP,Investment Grade Credit,Electric,Other,BBB,BBB,Utilities,00108WAR1,VCIT | VCIT (bench)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2624,NaN,USD,USD_CCASH,Unassigned,CASH,CASH,0.0,ORD,USD,0.000000,...,N,USD,NaN,Other,Other,NR,Junk / NR,NaN,USD_CCASH,VCIT | VCIT (bench)
2625,NaN,USD,USD_CCASH,Unassigned,CASH,CASH,0.0,TRD,USD,0.000000,...,N,USD,NaN,Other,Other,NR,Junk / NR,NaN,USD_CCASH,VCIT | VCIT (bench)
2626,NaN,USD,UXYZ42029,Unassigned,FUTURE,FIN,0.0,EOD,USD,5.619551,...,N,UXYZ4,NaN,Other,Other,AAA/AA,AAA,NaN,UXYZ42029,VCIT | VCIT (bench)
2627,NaN,USD,UXYZ4202F,Unassigned,CASH,FUTURE,0.0,EOD,USD,0.000000,...,N,UXYZ4,NaN,Other,Other,AAA/AA,AAA,NaN,UXYZ4202F,VCIT | VCIT (bench)


In [6]:
df_description = pd.read_excel("./data/1/data_assets_dictionary.xlsx", sheet_name="Dictionary")
df_description

,Field,Description
0,isin,12‑character International Securities Identifi...
1,ccy,ISO‑4217 currency in which the security is den...
2,assetId,Internal master identifier used by Vanguard sy...
3,strategyName,Friendly name of the trading or optimization s...
4,secGroup,"High‑level security grouping (Government, Corp..."
...,...,...
110,Large Issuer_new / issuerTicker_new,Post‑trade or scenario “what‑if” versions of l...
111,sodraw.filterLevel1_new … sodraw.filterLevel4_new,Scenario overrides of SoDraw filter levels 1–4.
112,security.elements.creditQualityBuckets_new / s...,Scenario overrides of credit‑quality bucket an...
113,assetId_new,Temporary scenario asset ID generated by the o...


Collect all 31 rows from `df` based on `binary_var_names`

In [7]:
pd.set_option("display.max_columns", 500)

In [8]:
df_31bonds = df[df["assetId"].isin(binary_var_names)]
df_31bonds.reset_index(inplace=True)

# df_31bonds_clean = df_31bonds.drop(["ccy", "strategyName", "secGroup", "secType", "posSource", 
#   "baseCcy", "krd15y", "krd20y", "krd25y", "krd30y", "krd40y", "krd50y",
#   "portfolioName",
#   "fund_orderStatus",
#   "enriched.cashladder_tCashBalance",
#   "enriched.cashladder_tPlus1CashBalance",
#   "enriched.cashladder_tPlus2CashBalance",
#   "enriched.cashladder_tPlus3CashBalance",
#   "enriched.cashladder_tPlus4CashBalance",
#   "enriched.cashladder_tPlus5CashBalance",
#   "enriched.cashladder_tPlus6CashBalance",
#   "enriched.cashladder_tPlus7CashBalance",
#   "enriched.cashladder_tPlus30CashBalance",
#   "enriched.cashladder_tPlus60CashBalance",
#   "enriched.cashladder_tPlus90CashBalance",
#   "bvb.krd10yCtr",
#   "bvb.krd15yCtr",
#   "bvb.krd20yCtr",
#   "bvb.krd25yCtr",
#   "bvb.krd30yCtr",
#   "bvb.krd40yCtr",
#   "bvb.krd50yCtr",
#   "trades.ctrkrd10y",
#   "trades.ctrkrd15y",
#   "trades.ctrkrd20y",
#   "trades.ctrkrd25y",
#   "trades.ctrkrd30y",
#   "trades.ctrkrd40y",
#   "trades.ctrkrd50y",
#   "fund_enriched.krd15yCtr",
#   "fund_enriched.krd20yCtr",
#   "fund_enriched.krd25yCtr",
#   "fund_enriched.krd30yCtr",
#   "fund_enriched.krd40yCtr",
#   "fund_enriched.krd50yCtr",
#   "enriched.krd15yCtr_active",
#   "enriched.krd20yCtr_active",
#   "enriched.krd25yCtr_active",
#   "enriched.krd30yCtr_active",
#   "enriched.krd40yCtr_active",
#   "enriched.krd50yCtr_active",
#   "fund_enriched.fund.krd15yCtr",
#   "fund_enriched.fund.krd20yCtr",
#   "fund_enriched.fund.krd25yCtr",
#   "fund_enriched.fund.krd30yCtr",
#   "fund_enriched.fund.krd40yCtr",
#   "fund_enriched.fund.krd50yCtr",
#   "sodraw.filterLevel1",
#   "sodraw.filterLevel1_new",
#   "sodraw.structure",
#   "sodraw.pathVngcoreCCYFilterLevel1",
#   "sodraw.pathVngcoreCCYFilterLevel2",
#   "sodraw.pathVngcoreCCYFilterLevel3",
#   "sodraw.pathVngcoreCCYFilterLevel4",
#   "sodraw.pathHUBGLOBNOSTRAT1",
#   "enriched.corpActions.usageType",
#   "enriched.tsyRV.bb_spread_to_spline_cubic",
#   "enriched.tsyRV.bb_3m_zscore_sp_spline_cubic",
#   "enriched.tsyRV.spline_range",
#   "enriched.tsyRV.rtn",
#   "enriched.tsyRV.expected_move",
#   "enriched.tsyRV.asset_swap_spd_mid",
#   "enriched.tsyRV.asw_range",
#   "enriched.otr_roll.SpecialStatusCTD",
#   "enriched.otr_roll.SpecialStatusOTR",
#   "enriched.otr_roll.OTR",
#   "bench_enriched.krd15yCtr",
#   "bench_enriched.krd20yCtr",
#   "bench_enriched.krd25yCtr",
#   "bench_enriched.krd30yCtr",
#   "bench_enriched.krd40yCtr",
#   "bench_enriched.krd50yCtr",
#   "main_key"], axis=1)

# df_31bonds_clean.to_excel("./clean_data/31bonds_assets.xlsx")

source_sheets = {
  "Data": df_31bonds,
  "Dictionary": df_description
}

with pd.ExcelWriter("./clean_data/31bonds_assets.xlsx", engine="openpyxl") as writer:
  for sheet_name, df in source_sheets.items():
    df.to_excel(writer, sheet_name=sheet_name, index=False)



In [204]:
for var_i, var_j in zip(df_31bonds["assetId"].to_list(), binary_var_names):
  print(var_i, var_j, var_i==var_j)

020002BJ9 020002BJ9 True
026874DS3 026874DS3 True
081437AT2 081437AT2 True
097023CJ2 097023CJ2 True
13645RAD6 13645RAD6 True
13645RBF0 13645RBF0 True
14448CBC7 14448CBC7 True
15135BAW1 15135BAW1 True
21871XAS8 21871XAS8 True
24422EWZ8 24422EWZ8 True
24422EXP9 24422EXP9 True
314353AA1 314353AA1 True
36166NAK9 36166NAK9 True
438516CM6 438516CM6 True
443201AC2 443201AC2 True
444859BR2 444859BR2 True
444859BV3 444859BV3 True
444859BY7 444859BY7 True
444859CA8 444859CA8 True
45687VAB2 45687VAB2 True
539830CD9 539830CD9 True
540424AT5 540424AT5 True
56501RAN6 56501RAN6 True
655844CR7 655844CR7 True
655844CT3 655844CT3 True
75513EAD3 75513EAD3 True
759351AP4 759351AP4 True
760759BA7 760759BA7 True
760759BC3 760759BC3 True
907818FX1 907818FX1 True
91324PEJ7 91324PEJ7 True


Breakdown the formula in `.lp` file

In [18]:
sodraw_filterLevel3_unique = df_31bonds["sodraw.filterLevel3"].unique()
sodraw_filterLevel3_unique

assetId_to_sodraw_filterLevel3 = {k: [] for k in sodraw_filterLevel3_unique}
for category in sodraw_filterLevel3_unique:
  assetId = df_31bonds[df_31bonds["sodraw.filterLevel3"] == category]["assetId"].to_list()
  assetId_to_sodraw_filterLevel3[category].append(assetId)

assetId_to_sodraw_filterLevel3


{'Insurance': [['020002BJ9',
   '026874DS3',
   '15135BAW1',
   '21871XAS8',
   '444859BR2',
   '444859BV3',
   '444859BY7',
   '444859CA8',
   '540424AT5',
   '56501RAN6',
   '759351AP4',
   '91324PEJ7']],
 'Capital Goods': [['081437AT2',
   '097023CJ2',
   '14448CBC7',
   '24422EWZ8',
   '24422EXP9',
   '36166NAK9',
   '438516CM6',
   '443201AC2',
   '45687VAB2',
   '539830CD9',
   '75513EAD3',
   '760759BA7',
   '760759BC3']],
 'Transportation': [['13645RAD6',
   '13645RBF0',
   '314353AA1',
   '655844CR7',
   '655844CT3',
   '907818FX1']]}

In [9]:
binary_var_names

['020002BJ9',
 '026874DS3',
 '081437AT2',
 '097023CJ2',
 '13645RAD6',
 '13645RBF0',
 '14448CBC7',
 '15135BAW1',
 '21871XAS8',
 '24422EWZ8',
 '24422EXP9',
 '314353AA1',
 '36166NAK9',
 '438516CM6',
 '443201AC2',
 '444859BR2',
 '444859BV3',
 '444859BY7',
 '444859CA8',
 '45687VAB2',
 '539830CD9',
 '540424AT5',
 '56501RAN6',
 '655844CR7',
 '655844CT3',
 '75513EAD3',
 '759351AP4',
 '760759BA7',
 '760759BC3',
 '907818FX1',
 '91324PEJ7']

In [44]:
alpha_i = np.sqrt(15.291443202948)
alpha_j = np.sqrt(22.817804115446)

alpha_i * alpha_j * 2

np.float64(37.35864856481485)

In [ ]:
selectedField_31bonds = df_31bonds[
  ["assetId", "cpn", "minTradeIncrement", "minTradeSize", 
   "fund_enriched.mktValue", "fund_enriched.pmv", 
   "fund_enriched.dxsCtr", "sodraw.filterLevel3", "price"]]

selectedField_31bonds_insurance = selectedField_31bonds[
  selectedField_31bonds["sodraw.filterLevel3"] == "Insurance"]
selectedField_31bonds_insurance

,assetId,cpn,minTradeIncrement,minTradeSize,fund_enriched.mktValue,fund_enriched.pmv,fund_enriched.dxsCtr,sodraw.filterLevel3,price
0,020002BJ9,1.450,1000.0,2000.0,1.569188e+07,0.000309,0.137962,Insurance,82.519348
1,026874DS3,5.125,1000.0,2000.0,2.455533e+07,0.000484,0.290454,Insurance,100.801844
7,15135BAW1,3.000,1000.0,2000.0,5.116108e+07,0.001008,0.783132,Insurance,87.254963
8,21871XAS8,5.750,1000.0,2000.0,2.349532e+07,0.000463,0.374742,Insurance,104.451479
15,444859BR2,2.150,1000.0,2000.0,1.951737e+07,0.000385,0.290703,Insurance,81.741299
16,444859BV3,5.875,1000.0,2000.0,2.404790e+07,0.000474,0.398056,Insurance,103.543152
17,444859BY7,5.950,1000.0,2000.0,2.269152e+07,0.000447,0.412717,Insurance,103.936975
18,444859CA8,5.375,1000.0,2000.0,3.067552e+07,0.000605,0.363540,Insurance,100.866503
21,540424AT5,3.200,1000.0,2000.0,1.253821e+07,0.000247,0.085637,Insurance,93.708560
22,56501RAN6,3.703,1000.0,2000.0,1.864972e+07,0.000368,0.161329,Insurance,93.426092


In [42]:
idx = 1
bond_i_fund_enriched_mktValue = selectedField_31bonds_insurance.iloc[idx][
  "fund_enriched.mktValue"]
bond_i_fund_enriched_pmv = selectedField_31bonds_insurance.iloc[idx]["fund_enriched.pmv"]
bond_i_price = selectedField_31bonds_insurance.iloc[idx]["price"]

print(bond_i_fund_enriched_mktValue)
print(bond_i_fund_enriched_pmv)
print(bond_i_price)
const = 1790.6

bond_i_price / 1790.6

24555329.11576667
0.0004839601109290001
100.8018436607827


np.float64(0.05629500930458098)

Objective 
$$
  \argmin_{y_{\ell, i}} \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \gamma_{\ell, i} y_{\ell, i}
  + \frac{1}{2}
    \left(\sum_{\ell\in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell} 
    (\alpha_{\ell,i} y_{\ell,i}) (\alpha_{\ell, j} y_{\ell,j}) \right)
  + C
$$

$$
  = \argmin_{y_{\ell, i}}
    \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \gamma_{\ell, i} y_{\ell, i}
  + 
  \frac{1}{2} \left(\sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \alpha_i^2 y_i^2
  + \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell \setminus \{i\}}
    2 \alpha_i \alpha_j y_i y_j \right)
  + C
$$

Constraints

`trade_num_upper.limit`
$$
  \left(\sum_{\ell \in L}  \sum_{i \in \mathbb{K}_\ell} y_{\ell, i} \right) \leq M
$$

`limit_sodraw.filterLevel1_sodraw.filterLevel3_#_fund_enriched.pmv`

$$
  K_{\ell}^\text{min} \leq 
    \sum_{i \in \mathbb{K}_{\ell}} \beta_{\ell, i} y_{\ell, i}
    \leq K_{\ell}^\text{max}, \qquad \forall \ell \in L
$$

`limt_sodrtaw.filterLevel1_sodraw.filterLevel3_#_fund_enriched.dxsCtr`

$$
  O_{\ell}^\text{min} \leq 
    \sum_{i \in \mathbb{K}_{\ell}} \varepsilon_{\ell, i} y_{\ell, i}
    \leq O_{\ell}^\text{max}, \qquad \forall \ell \in L
$$

`low_cnstr_cash`

$$
  P^\text{min} \leq \sum_{\ell \in L}  \sum_{i \in \mathbb{K}_\ell}
    \beta_{\ell, i} y_{\ell, i} \leq P^\text{max}
$$

$y_{\ell, i}$ are binary variables

$$
\begin{align*}
  L &= \{1, 2, 3\}  \\
    &= \{\text{Capital Goods}, \text{Insurance}, \text{Transporation}\}
\end{align*}
$$

$$
  \mathbb{K}_{\ell}
  = \begin{cases}
      \begin{aligned}
      \mathbb{K}_1 &= \{1, 2, 3, \ldots, 12\} 
        = \left\{
        \begin{array}{l}
        \texttt{020002BJ9}, \texttt{026874DS3}, \texttt{15135BAW1}, 
        \texttt{21871XAS8}, \texttt{444859BR2}, \\ \texttt{444859BV3}, 
        \texttt{444859BY7}, \texttt{444859CA8}, \texttt{540424AT5}, 
        \texttt{56501RAN6}, \\ \texttt{759351AP4}, \texttt{91324PEJ7} 
        \end{array}
        \right\},
      \end{aligned} \\[16pt]
      \mathbb{K}_2 = \{1, 2, 3, \ldots, 13\}
        = \left\{
        \begin{array}{l}
          \texttt{081437AT2}, \texttt{097023CJ2}, \texttt{14448CBC7}, 
          \texttt{24422EWZ8}, \texttt{24422EXP9}, \\ \texttt{36166NAK9}, 
          \texttt{438516CM6}, \texttt{443201AC2}, \texttt{45687VAB2}, 
          \texttt{539830CD9}, \\ \texttt{75513EAD3}, \texttt{760759BA7}, 
          \texttt{760759BC3}
        \end{array}
        \right\}, \\[16pt]
      \mathbb{K}_3 = \{1, 2, 3, \ldots, 6\}
        = \left\{
        \begin{array}{l}
          \texttt{13645RAD6}, \texttt{13645RBF0}, \texttt{314353AA1}, 
          \texttt{655844CR7}, \texttt{655844CT3}, \\ \texttt{907818FX1}
        \end{array}
        \right\}
    \end{cases}
$$

$$
  M = \texttt{trade\_num\_upper\_limit} = 500
$$

### Get all parameters 

In [50]:
def extract_objective_robust(lp_content):
  # Pattern that handles multi-line objectives
  pattern = r'obj:\s*(.+?)(?=\n\s*(?:subject\s+to|st|bounds|end|\Z))'
  
  match = re.search(pattern, lp_content, re.IGNORECASE | re.DOTALL)
  if match:
    # Clean up the expression (remove extra whitespace, newlines)
    expression = re.sub(r'\s+', ' ', match.group(1).strip())
    return expression
  return None

In [86]:
def get_coef_of_var(var_str, expr_str, suffix=" "):
  pattern = r"(-?\s+\d+\.?\d*)\s+" + "iTrade_" + var_str + suffix
  match = re.search(pattern, expr_str)
  number = float(match.group(1).replace(" ", ""))
  return number

Parse $\gamma_{\ell, i}$

In [87]:
with open(lp_file_path, "r") as lp_pointer:
  data = lp_pointer.read()

print(obj_expr_str)

gamma_arr = []
for var in binary_var_names:
  number = get_coef_of_var(var, obj_expr_str) 
  gamma_arr.append(number)

gamma_arr

- 171.843480141259 iTrade_020002BJ9 - 209.916098093165 iTrade_026874DS3 - 208.476006609091 iTrade_081437AT2 - 102.658455587925 iTrade_097023CJ2 - 108.872687135303 iTrade_13645RAD6 - 59.317668233044 iTrade_13645RBF0 - 248.173707501463 iTrade_14448CBC7 - 181.705222582624 iTrade_15135BAW1 - 217.516328228335 iTrade_21871XAS8 - 150.364227654476 iTrade_24422EWZ8 - 238.537754989087 iTrade_24422EXP9 - 64.315591692518 iTrade_314353AA1 - 232.716684573550 iTrade_36166NAK9 - 231.561240438637 iTrade_438516CM6 - 133.172276708478 iTrade_443201AC2 - 111.860973316693 iTrade_444859BR2 - 215.624770871618 iTrade_444859BV3 - 216.444892837478 iTrade_444859BY7 - 210.050748767208 iTrade_444859CA8 - 98.562779686790 iTrade_45687VAB2 - 234.562738134276 iTrade_539830CD9 - 195.144598835887 iTrade_540424AT5 - 194.556369478498 iTrade_56501RAN6 - 190.182474291791 iTrade_655844CR7 - 26.345955030805 iTrade_655844CT3 - 188.770788790166 iTrade_75513EAD3 - 192.768933140209 iTrade_759351AP4 - 193.225791754691 iTrade_760759

[-171.843480141259,
 -209.916098093165,
 -208.476006609091,
 -102.658455587925,
 -108.872687135303,
 -59.317668233044,
 -248.173707501463,
 -181.705222582624,
 -217.516328228335,
 -150.364227654476,
 -238.537754989087,
 -64.315591692518,
 -232.71668457355,
 -231.561240438637,
 -133.172276708478,
 -111.860973316693,
 -215.624770871618,
 -216.444892837478,
 -210.050748767208,
 -98.56277968679,
 -234.562738134276,
 -195.144598835887,
 -194.556369478498,
 -190.182474291791,
 -26.345955030805,
 -188.770788790166,
 -192.768933140209,
 -193.225791754691,
 -234.355342136515,
 -85.550380002185,
 -152.218422802645]

Parse $\alpha_{\ell, i}$

In [120]:
pattern = r'\[([^\]]*)\]'
match = re.search(pattern, obj_expr_str)
mixed_term = match.group(1)
display(mixed_term)

alpha_arr = []
for var in binary_var_names:
  number = get_coef_of_var(var + r"\^2", mixed_term, suffix="")
  alpha_arr.append(float(np.sqrt(number))) 

alpha_arr

' 15.291443202948 iTrade_020002BJ9^2 + 37.358648564815 iTrade_020002BJ9*iTrade_026874DS3 + 32.337975098236 iTrade_020002BJ9*iTrade_15135BAW1 + 38.711257198506 iTrade_020002BJ9*iTrade_21871XAS8 + 19.907833787963 iTrade_020002BJ9*iTrade_444859BR2 + 38.374617811762 iTrade_020002BJ9*iTrade_444859BV3 + 38.520574451491 iTrade_020002BJ9*iTrade_444859BY7 + 37.382612268677 iTrade_020002BJ9*iTrade_444859CA8 + 34.729773244909 iTrade_020002BJ9*iTrade_540424AT5 + 34.625086400795 iTrade_020002BJ9*iTrade_56501RAN6 + 34.306977372470 iTrade_020002BJ9*iTrade_759351AP4 + 27.090226115248 iTrade_020002BJ9*iTrade_91324PEJ7 + 22.817804115446 iTrade_026874DS3^2 + 39.502584254437 iTrade_026874DS3*iTrade_15135BAW1 + 47.287892777261 iTrade_026874DS3*iTrade_21871XAS8 + 24.318494869990 iTrade_026874DS3*iTrade_444859BR2 + 46.876669573031 iTrade_026874DS3*iTrade_444859BV3 + 47.054963496534 iTrade_026874DS3*iTrade_444859BY7 + 45.664881190254 iTrade_026874DS3*iTrade_444859CA8 + 42.424294952819 iTrade_026874DS3*iTrade_

[3.9104274961886203,
 4.776798521546204,
 4.233645398427341,
 2.084745890803721,
 9.988283313561595,
 5.4419679664485345,
 5.039810057108105,
 4.134838854544152,
 4.949747468305833,
 3.053535180626875,
 4.8441270781551555,
 5.900491373983103,
 4.725915163127137,
 4.702450876639223,
 2.704408078681729,
 2.54548048868912,
 4.906703659531417,
 4.925366151020145,
 4.779862598796853,
 2.0015725812498033,
 4.763404063119672,
 4.440661958156689,
 4.427276357194003,
 17.447869474736134,
 2.4170512360643084,
 3.833479901682543,
 4.386601900419846,
 3.923950278142296,
 4.7591923500988065,
 7.848629950528691,
 3.4638445721922917]

Parse $C$

In [123]:
C_num = float(obj_expr_str.split("+")[-1])
C_num

2237.404382444853

Parse $M$

In [151]:
model = ModelReader.read_model(lp_file_path)
constraints = list(model.iter_constraints())

# --- important constraint attributes
# .name, .left_expr, .sense, .rhs

M_const = constraints[0].rhs
M_const = M_const._raw_solution_value()
M_const

500.0

Parse $\beta_{\ell, i}$ and $\varepsilon_{\ell, i}$

In [205]:
print(sodraw_filterLevel3_unique)

['Insurance' 'Capital Goods' 'Transportation']


In [215]:
assetId_to_pmv_min_coeffs = {}
assetId_to_dxsCtr_min_coeffs = {}
for constraint in constraints:
  if "pmv_min" in constraint.name:
    print(constraint.name)

    left_expr = constraint.left_expr
    for var in left_expr.iter_variables():
      assetId_to_pmv_min_coeffs[var.name.split("_")[-1]] = left_expr.get_coef(var)
  
  elif "dxsCtr_min" in constraint.name:
    print(constraint.name)

    left_expr = constraint.left_expr
    for var in left_expr.iter_variables():
      assetId_to_dxsCtr_min_coeffs[var.name.split("_")[-1]] = left_expr.get_coef(var)

# display(assetId_to_pmv_min_coeffs)

beta_arr = []
epsilon_arr = []
for var in binary_var_names:
  beta_arr.append(assetId_to_pmv_min_coeffs[var])
  epsilon_arr.append(assetId_to_dxsCtr_min_coeffs[var])

print(f"{'var':^9s} {'beta_i':^14s} {'epsilon_i':^16s}")
for var, beta_i, epsilon_i in zip(binary_var_names, beta_arr, epsilon_arr):
  print(f"{var} {beta_i} {epsilon_i:>16.12f}")

limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Capital_Goods_fund_enriched.pmv_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Capital_Goods_fund_enriched.dxsCtr_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Insurance_fund_enriched.pmv_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Insurance_fund_enriched.dxsCtr_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Transportation_fund_enriched.pmv_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Transportation_fund_enriched.dxsCtr_min
   var        beta_i        epsilon_i    
020002BJ9 0.460848299982 205.579590690186
026874DS3 0.562951104491 337.861841810534
081437AT2 0.498939895061 233.033365950620
097023CJ2 0.245689659406 310.153535258368
13645RAD6 1.177130477239 490.408171768948
13645RBF0 0.641342075346 242.412768936651
14448CBC7 0.593947311212 393.282773577898
15135BAW1 0.487295432194 

Parse $K_{\ell}^\text{min}$, $K_{\ell}^\text{max}$, 
$O_{\ell}^\text{min}$, $O_{\ell}^\text{max}$, $P^\text{min}$, $P^\text{max}$

In [218]:
sodraw_filterLevel3_unique

array(['Insurance', 'Capital Goods', 'Transportation'], dtype=object)

$$
  L = \{1, 2, 3\} = \{\text{Capital Goods}, \text{Insurance}, \text{Transportation}\}
$$

In [216]:
for constraint in constraints:
  print(constraint.name)

trade_num_upper_limit
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Capital_Goods_fund_enriched.pmv_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Capital_Goods_fund_enriched.pmv_max
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Capital_Goods_fund_enriched.dxsCtr_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Capital_Goods_fund_enriched.dxsCtr_max
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Insurance_fund_enriched.pmv_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Insurance_fund_enriched.pmv_max
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Insurance_fund_enriched.dxsCtr_min
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Insurance_fund_enriched.dxsCtr_max
limit_sodraw.filterLevel1_sodraw.filterLevel3_Investment_Grade_Credit_Transportation_fund_enriched.pmv_min
limit_sodraw.filterLevel1_s

In [230]:
K_min_dict = {
  "Capital Goods": constraints[1].rhs._raw_solution_value(),
  "Insurance": constraints[5].rhs._raw_solution_value(),
  "Transportation": constraints[9].rhs._raw_solution_value()}
K_max_dict = {
  "Capital Goods": constraints[2].rhs._raw_solution_value(),
  "Insurance": constraints[6].rhs._raw_solution_value(),
  "Transportation": constraints[10].rhs._raw_solution_value()}

O_min_dict = {
  "Capital Goods": constraints[3].rhs._raw_solution_value(),
  "Insurance": constraints[7].rhs._raw_solution_value(),
  "Transportation": constraints[11].rhs._raw_solution_value()}
O_max_dict = {
  "Capital Goods": constraints[4].rhs._raw_solution_value(),
  "Insurance": constraints[8].rhs._raw_solution_value(),
  "Transportation": constraints[12].rhs._raw_solution_value()}

P_min = constraints[13].rhs._raw_solution_value()
P_max = constraints[14].rhs._raw_solution_value()

K_min_dict, K_max_dict, O_min_dict, O_max_dict, P_min, P_max

({'Capital Goods': 4.758736462913,
  'Insurance': 4.134392251914,
  'Transportation': 0.240014083139},
 {'Capital Goods': 6.847872350911,
  'Insurance': 6.223528139912,
  'Transportation': 2.329149971137},
 {'Capital Goods': 1780.68926045072,
  'Insurance': 1991.42972820855,
  'Transportation': -479.881210498972},
 {'Capital Goods': 3869.825148448374,
  'Insurance': 4080.565616206205,
  'Transportation': 1609.254677498683},
 2.600974180557,
 10.435233760548)

Store into `.json`

In [242]:
var_params_31bonds_dict = {
  "var": binary_var_names,
  "gamma": gamma_arr, 
  "alpha": alpha_arr,
  "C": C_num,
  "M": M_const,
  "beta": beta_arr,
  "epsilon": epsilon_arr,
  "K_min": K_min_dict, 
  "K_max": K_max_dict, 
  "O_min": O_min_dict,
  "O_max": O_max_dict,
  "P_min": P_min,
  "P_max": P_max
}

var_params_31bonds_dict

{'var': ['020002BJ9',
  '026874DS3',
  '081437AT2',
  '097023CJ2',
  '13645RAD6',
  '13645RBF0',
  '14448CBC7',
  '15135BAW1',
  '21871XAS8',
  '24422EWZ8',
  '24422EXP9',
  '314353AA1',
  '36166NAK9',
  '438516CM6',
  '443201AC2',
  '444859BR2',
  '444859BV3',
  '444859BY7',
  '444859CA8',
  '45687VAB2',
  '539830CD9',
  '540424AT5',
  '56501RAN6',
  '655844CR7',
  '655844CT3',
  '75513EAD3',
  '759351AP4',
  '760759BA7',
  '760759BC3',
  '907818FX1',
  '91324PEJ7'],
 'gamma': [-171.843480141259,
  -209.916098093165,
  -208.476006609091,
  -102.658455587925,
  -108.872687135303,
  -59.317668233044,
  -248.173707501463,
  -181.705222582624,
  -217.516328228335,
  -150.364227654476,
  -238.537754989087,
  -64.315591692518,
  -232.71668457355,
  -231.561240438637,
  -133.172276708478,
  -111.860973316693,
  -215.624770871618,
  -216.444892837478,
  -210.050748767208,
  -98.56277968679,
  -234.562738134276,
  -195.144598835887,
  -194.556369478498,
  -190.182474291791,
  -26.345955030805,

In [245]:
with open("./data/1/31bonds/docplex-bin-avgonly-nocplexvars.json", "w") as fp:
  json.dump(var_params_31bonds_dict, fp, indent=2)